In [ ]:
# 📌 Instalar dependencias (si no están instaladas)
!pip install nltk scikit-learn

In [ ]:
import json
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# 📌 Descargar datos de NLTK (solo la primera vez)
nltk.download('stopwords')
nltk.download('wordnet')

# 📌 Configuración de NLP
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# 📌 Cargar el dataset limpio desde el JSON
with open('celulares_limpios.json', 'r') as file:
    celulares = json.load(file)

# 📌 Mostrar los primeros 5 celulares para verificar los datos
celulares[:5]

# 📌 Definir intenciones con palabras clave
intenciones = {
    "gaming": ["gaming", "juegos", "fps", "alto rendimiento", "potente"],
    "fotografía": ["cámara", "fotografía", "selfie", "megapíxeles", "foto"],
    "batería": ["batería", "duración", "mAh", "autonomía"],
    "económico": ["barato", "económico", "asequible", "accesible"],
    "alta_gama": ["gama alta", "flagship", "premium", "exclusivo"]
}

# 📌 Función para detectar intención en la consulta del usuario
def detectar_intencion(texto):
    texto = texto.lower()
    for intencion, palabras in intenciones.items():
        for palabra in palabras:
            if re.search(rf"\b{palabra}\b", texto):
                return intencion
    return "desconocida"

# 📌 Función para limpiar texto y eliminar palabras irrelevantes
def limpiar_texto(texto):
    tokens = texto.lower().split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# 📌 Aplicamos limpieza a los nombres de procesadores
for celular in celulares:
    celular["Processor"] = limpiar_texto(celular["Processor"])

# 📌 Vectorización de los procesadores con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([celular["Processor"] for celular in celulares])

# 📌 Extraer valores numéricos para normalizar
ram_values = np.array([cel["RAM (GB)"] for cel in celulares]).reshape(-1, 1)
battery_values = np.array([cel["Battery Capacity (mAh)"] for cel in celulares]).reshape(-1, 1)
price_values = np.array([cel["Launched Price (USA)"] for cel in celulares]).reshape(-1, 1)

# 📌 Normalizar los valores entre 0 y 1
scaler = MinMaxScaler()
ram_scaled = scaler.fit_transform(ram_values)
battery_scaled = scaler.fit_transform(battery_values)
price_scaled = scaler.fit_transform(price_values)

# 📌 Agregar los valores normalizados al dataset
for i, cel in enumerate(celulares):
    cel["RAM_scaled"] = ram_scaled[i][0]
    cel["Battery_scaled"] = battery_scaled[i][0]
    cel["Price_scaled"] = price_scaled[i][0]

def recomendar_celular(criterio):
    # 📌 Detectar la intención del usuario
    intencion = detectar_intencion(criterio)

    # 📌 Filtrar celulares según la intención
    if intencion == "gaming":
        celulares_filtrados = [c for c in celulares if c["RAM (GB)"] >= 8]  # Gaming necesita más de 8GB RAM
    elif intencion == "fotografía":
        celulares_filtrados = [c for c in celulares if c["Back Camera (MP)"] and c["Back Camera (MP)"] >= 50]  # Cámara potente
    elif intencion == "batería":
        celulares_filtrados = [c for c in celulares if c["Battery Capacity (mAh)"] >= 5000]  # Batería grande
    elif intencion == "económico":
        celulares_filtrados = sorted(celulares, key=lambda x: x["Launched Price (USA)"])[:10]  # Los 10 más baratos
    elif intencion == "alta_gama":
        celulares_filtrados = [c for c in celulares if c["Launched Price (USA)"] >= 900]  # Celulares premium
    else:
        celulares_filtrados = celulares  # Si la intención no se detecta, usar todos

    # 📌 Calcular similitud de coseno basada en procesador
    criterio_vector = vectorizer.transform([limpiar_texto(criterio)])
    similitudes = cosine_similarity(criterio_vector, tfidf_matrix).flatten()

    # 📌 Crear una puntuación combinada con características numéricas
    mejores_puntajes = []
    
    for cel in celulares_filtrados:
        i = celulares.index(cel)  # Obtener índice original

        # 📌 Calcular diferencias en atributos numéricos
        diff_ram = abs(cel["RAM_scaled"] - 1)  # Preferimos RAM alta
        diff_battery = abs(cel["Battery_scaled"] - 1)  # Preferimos batería alta
        diff_price = abs(cel["Price_scaled"] - 0)  # Preferimos precio bajo
        
        # 📌 Ponderar cada factor según la intención
        if intencion == "gaming":
            score = (similitudes[i] * 0.5) + ((1 - diff_ram) * 0.3) + ((1 - diff_battery) * 0.15) + ((1 - diff_price) * 0.05)
        elif intencion == "fotografía":
            score = (similitudes[i] * 0.4) + ((1 - diff_battery) * 0.2) + ((cel["Back Camera (MP)"] / 108) * 0.4)  # Peso fuerte en la cámara
        elif intencion == "batería":
            score = (similitudes[i] * 0.3) + ((1 - diff_battery) * 0.6) + ((1 - diff_ram) * 0.1)
        elif intencion == "económico":
            score = (similitudes[i] * 0.2) + ((1 - diff_price) * 0.7) + ((1 - diff_battery) * 0.1)
        elif intencion == "alta_gama":
            score = (similitudes[i] * 0.5) + ((1 - diff_ram) * 0.2) + ((1 - diff_battery) * 0.2) + ((cel["Launched Price (USA)"] / 2000) * 0.1)
        else:
            score = (similitudes[i] * 0.6) + ((1 - diff_ram) * 0.2) + ((1 - diff_battery) * 0.15) + ((1 - diff_price) * 0.05)

        mejores_puntajes.append((score, cel))

    # 📌 Elegir el mejor celular basado en la puntuación final
    mejor_celular = max(mejores_puntajes, key=lambda x: x[0])[1]

    return f"📱 Te recomiendo el {mejor_celular['Company Name']} {mejor_celular['Model Name']} con {mejor_celular['RAM (GB)']}GB RAM, {mejor_celular['Battery Capacity (mAh)']}mAh y procesador {mejor_celular['Processor']} por ${mejor_celular['Launched Price (USA)']}."

pregunta = input("👤 Escribe tu consulta: ")
intencion = detectar_intencion(pregunta)
print(f"🔍 Intención detectada: {intencion}")
print(recomendar_celular(pregunta))